## The goal: to web scrape the remaining brands not on sizecharts.com and add it to my universe of data. 
### The universe of data is defined in my ReadMe.md. 

### Let's scrape Amazon's website. 

In [1]:
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [2]:
from playwright.async_api import async_playwright

In [13]:
await page.goto("https://z-ecx.images-amazon.com/images/G/01/apparel/en_US/html/sizechart/womens._V154091249_.html")

<Response url='https://z-ecx.images-amazon.com/images/G/01/apparel/en_US/html/sizechart/womens._V154091249_.html' request=<Request url='https://z-ecx.images-amazon.com/images/G/01/apparel/en_US/html/sizechart/womens._V154091249_.html' method='GET'>>

In [14]:
import io
import pandas as pd

html = await page.content()
tables = pd.read_html(io.StringIO(html))
df = tables[0]
df

,0,1,2,3,4
0,We do not have specific size information for t...,NaN,NaN,NaN,NaN
1,General Size,US Clothing Size,Chest/Bust Measurement (in inches),Waist Measurement (in inches),"Hips (8"" from waist)"
2,XS,0 2,32 33,24 25,34.5 35.5
3,S,4 6,34 35,26 27,36.5 37.5
4,M,8 10,36 37,28 29,38.5 39.5
5,L,12 14,38.5 40,30.5 32,41 42.5
6,XL,16 18,41.5 43,33.5 35,44 45.5
7,General Size,US Clothing Size,Chest/Bust Measurement (in inches),Waist Measurement (in inches),"Hips (8"" from waist)"
8,0X,12W,40,33,42
9,1X,14W 16W,42 44,35 37,44 46


### Remove the the first row and then, bring that first row into a column. 

In [15]:
df = df.iloc[1:]     
df.columns = df.iloc[0]  
df = df.iloc[1:6].reset_index(drop=True)  
df

1,General Size,US Clothing Size,Chest/Bust Measurement (in inches),Waist Measurement (in inches),"Hips (8"" from waist)"
0,XS,0 2,32 33,24 25,34.5 35.5
1,S,4 6,34 35,26 27,36.5 37.5
2,M,8 10,36 37,28 29,38.5 39.5
3,L,12 14,38.5 40,30.5 32,41 42.5
4,XL,16 18,41.5 43,33.5 35,44 45.5


In [16]:
import pandas as pd
import numpy as np

rows = []
for idx, row in df.iterrows():
    row1 = [row['General Size'], row['US Clothing Size'].split()[0]]
    row1 += [col.split()[0] for col in row[2:]]
    
    row2 = [row['General Size'], row['US Clothing Size'].split()[1]]
    row2 += [col.split()[1] for col in row[2:]]
    
    rows.append(row1)
    rows.append(row2)

df_amazon = pd.DataFrame(rows, columns=df.columns)
df_amazon

1,General Size,US Clothing Size,Chest/Bust Measurement (in inches),Waist Measurement (in inches),"Hips (8"" from waist)"
0,XS,0,32,24,34.5
1,XS,2,33,25,35.5
2,S,4,34,26,36.5
3,S,6,35,27,37.5
4,M,8,36,28,38.5
5,M,10,37,29,39.5
6,L,12,38.5,30.5,41
7,L,14,40,32,42.5
8,XL,16,41.5,33.5,44
9,XL,18,43,35,45.5


## Let's ensure these columns are floats not strings. 

In [17]:
cols = [
    'US Clothing Size',
    'Chest/Bust Measurement (in inches)',
    'Waist Measurement (in inches)',
    'Hips (8" from waist)'
]

df_amazon[cols] = df_amazon[cols].astype(float)

df_amazon = df_amazon.drop(['US Clothing Size'], axis=1)

In [109]:
df_amazon= df_amazon.rename(columns={
    'Chest/Bust Measurement (in inches)': 'Bust',
    'Waist Measurement (in inches)': 'Waist',
    'Hips (8" from waist)' : 'Hip'
})

df_amazon['Brand']='Amazon'
df_amazon

1,General Size,Bust,Waist,Hip,Brand
0,XS,32.0,24.0,34.5,Amazon
1,XS,33.0,25.0,35.5,Amazon
2,S,34.0,26.0,36.5,Amazon
3,S,35.0,27.0,37.5,Amazon
4,M,36.0,28.0,38.5,Amazon
5,M,37.0,29.0,39.5,Amazon
6,L,38.5,30.5,41.0,Amazon
7,L,40.0,32.0,42.5,Amazon
8,XL,41.5,33.5,44.0,Amazon
9,XL,43.0,35.0,45.5,Amazon


## Let's scrape Target's website. 

In [28]:
await page.goto("https://digitalcontent.target.com/itemcontent/sizecharts/htmlfragments/women-clothing/a-new-day/womens.html")

<Response url='https://digitalcontent.target.com/itemcontent/sizecharts/htmlfragments/women-clothing/a-new-day/womens.html' request=<Request url='https://digitalcontent.target.com/itemcontent/sizecharts/htmlfragments/women-clothing/a-new-day/womens.html' method='GET'>>

In [29]:
import io
import pandas as pd

html = await page.content()
tables = pd.read_html(io.StringIO(html))
df = tables[0]
df

,Size,Size.1,Bust (in),Waist (in),Hip (in)
0,XXS,0,32.0,25.0,34.00
1,XS,0,33.0,26.0,35.00
2,XS,2,34.0,27.0,36.00
3,S,4,35.0,28.0,37.00
4,S,6,36.0,29.0,38.00
5,M,8,37.0,30.5,39.50
6,M,10,38.5,32.0,41.00
7,L,12,40.0,34.0,42.50
8,L,14,41.5,36.0,44.00
9,XL,16,43.5,38.0,46.00


## Only keep from XS to XL. 

In [30]:
df = df.iloc[2:]     
df

,Size,Size.1,Bust (in),Waist (in),Hip (in)
2,XS,2,34.0,27.0,36.00
3,S,4,35.0,28.0,37.00
4,S,6,36.0,29.0,38.00
5,M,8,37.0,30.5,39.50
6,M,10,38.5,32.0,41.00
7,L,12,40.0,34.0,42.50
8,L,14,41.5,36.0,44.00
9,XL,16,43.5,38.0,46.00
10,XXL,17,45.5,40.5,48.25
11,1X,18,47.5,43.0,50.50


In [31]:
df_target = df.iloc[0:8].reset_index(drop=True)  
df_target

,Size,Size.1,Bust (in),Waist (in),Hip (in)
0,XS,2,34.0,27.0,36.0
1,S,4,35.0,28.0,37.0
2,S,6,36.0,29.0,38.0
3,M,8,37.0,30.5,39.5
4,M,10,38.5,32.0,41.0
5,L,12,40.0,34.0,42.5
6,L,14,41.5,36.0,44.0
7,XL,16,43.5,38.0,46.0


In [32]:
df_target = df_target.rename(columns={
    'Size': 'General Size',
    'Size.1': 'US Clothing Size',
    'Bust (in)': 'Bust',
    'Waist (in)': 'Waist',
    'Hip (in)' : 'Hip'
})
df_target

,General Size,US Clothing Size,Bust,Waist,Hip
0,XS,2,34.0,27.0,36.0
1,S,4,35.0,28.0,37.0
2,S,6,36.0,29.0,38.0
3,M,8,37.0,30.5,39.5
4,M,10,38.5,32.0,41.0
5,L,12,40.0,34.0,42.5
6,L,14,41.5,36.0,44.0
7,XL,16,43.5,38.0,46.0


In [ ]:
df_target = df_target.drop(['US Clothing Size'], axis=1)
df_target['Brand'] = 'Target'
df_target

## Let's scrape Garage's website

In [35]:
await page.goto("https://www.garageclothing.com/us/gar-size-chart.html?srsltid=AfmBOoqGWZIX-2jt4JfI6UBnYhkXA8SowQkD1DLp8EwowWWDRnGEbvP0")

<Response url='https://www.garageclothing.com/us/gar-size-chart.html?srsltid=AfmBOoqGWZIX-2jt4JfI6UBnYhkXA8SowQkD1DLp8EwowWWDRnGEbvP0' request=<Request url='https://www.garageclothing.com/us/gar-size-chart.html?srsltid=AfmBOoqGWZIX-2jt4JfI6UBnYhkXA8SowQkD1DLp8EwowWWDRnGEbvP0' method='GET'>>

In [36]:
import io
import pandas as pd

html = await page.content()
tables = pd.read_html(io.StringIO(html))
df = tables[0]
df

,Size,Double Size,Number Size,Bust,Natural Waist,Low Hips
0,XXS,-,0,31”,23.5”,33.5”
1,XXS,-,0,32”,24.5”,34.5”
2,XS,XS/S,1,33”,25.5”,35.5”
3,XS,XS/S,3,34”,26.5”,36.5”
4,S,XS/S,5,35”,27.5”,37.5”
5,M,M/L,7,36”,28.5”,38.5”
6,M,M/L,9,37.5”,30”,40”
7,L,M/L,11,39”,31.5”,41.5”
8,L,M/L,13,40.5”,33”,43”
9,XL,-,15,42”,34.5”,44.5”


## Cleaning the df. 

In [37]:
df = df.iloc[2:]     
df

,Size,Double Size,Number Size,Bust,Natural Waist,Low Hips
2,XS,XS/S,1,33”,25.5”,35.5”
3,XS,XS/S,3,34”,26.5”,36.5”
4,S,XS/S,5,35”,27.5”,37.5”
5,M,M/L,7,36”,28.5”,38.5”
6,M,M/L,9,37.5”,30”,40”
7,L,M/L,11,39”,31.5”,41.5”
8,L,M/L,13,40.5”,33”,43”
9,XL,-,15,42”,34.5”,44.5”
10,XL,-,17,43.5”,36”,46”
11,XXL,-,19,45”,37.5”,47.5”


## We are only interested in XS to XL. 

In [38]:
df_garage = df.iloc[0:8].reset_index(drop=True)  
df_garage

,Size,Double Size,Number Size,Bust,Natural Waist,Low Hips
0,XS,XS/S,1,33”,25.5”,35.5”
1,XS,XS/S,3,34”,26.5”,36.5”
2,S,XS/S,5,35”,27.5”,37.5”
3,M,M/L,7,36”,28.5”,38.5”
4,M,M/L,9,37.5”,30”,40”
5,L,M/L,11,39”,31.5”,41.5”
6,L,M/L,13,40.5”,33”,43”
7,XL,-,15,42”,34.5”,44.5”


In [39]:
df_garage = df_garage.rename(columns={
    'Size': 'General Size',
    'Number Size': 'US Clothing Size',
    'Natural Waist': 'Waist',
    'Low Hips' : 'Hip'
})

df_garage

,General Size,Double Size,US Clothing Size,Bust,Waist,Hip
0,XS,XS/S,1,33”,25.5”,35.5”
1,XS,XS/S,3,34”,26.5”,36.5”
2,S,XS/S,5,35”,27.5”,37.5”
3,M,M/L,7,36”,28.5”,38.5”
4,M,M/L,9,37.5”,30”,40”
5,L,M/L,11,39”,31.5”,41.5”
6,L,M/L,13,40.5”,33”,43”
7,XL,-,15,42”,34.5”,44.5”


## Convert these columns to integers/numeric

In [40]:
cols_to_convert = [
    'Bust',
    'Waist',
    'Hip'
]

for col in cols_to_convert:
    df_garage[col] = pd.to_numeric(df_garage[col].str.replace(r'[^0-9.]', '', regex=True), errors='coerce')
df_garage

,General Size,Double Size,US Clothing Size,Bust,Waist,Hip
0,XS,XS/S,1,33.0,25.5,35.5
1,XS,XS/S,3,34.0,26.5,36.5
2,S,XS/S,5,35.0,27.5,37.5
3,M,M/L,7,36.0,28.5,38.5
4,M,M/L,9,37.5,30.0,40.0
5,L,M/L,11,39.0,31.5,41.5
6,L,M/L,13,40.5,33.0,43.0
7,XL,-,15,42.0,34.5,44.5


## Add Brand Garage. 

In [41]:
df_garage['Brand'] = 'Garage'
df_garage = df_garage.drop(['US Clothing Size'], axis=1)
df_garage

,General Size,Double Size,Bust,Waist,Hip,Brand
0,XS,XS/S,33.0,25.5,35.5,Garage
1,XS,XS/S,34.0,26.5,36.5,Garage
2,S,XS/S,35.0,27.5,37.5,Garage
3,M,M/L,36.0,28.5,38.5,Garage
4,M,M/L,37.5,30.0,40.0,Garage
5,L,M/L,39.0,31.5,41.5,Garage
6,L,M/L,40.5,33.0,43.0,Garage
7,XL,-,42.0,34.5,44.5,Garage


## Let's scrape Mango 

In [42]:
await page.goto("https://shop.mango.com/us/en/size-guide")

<Response url='https://shop.mango.com/us/en/size-guide' request=<Request url='https://shop.mango.com/us/en/size-guide' method='GET'>>

In [43]:
import io
import pandas as pd

html = await page.content()
tables = pd.read_html(io.StringIO(html))
df = tables[0]
df

,Size,Bust,Waist,Hip
0,0,30.71,23.23,33.86
1,2,32.28,24.41,35.43
2,4,33.86,25.98,37.01
3,6,36.22,28.35,39.37
4,8,38.58,30.71,41.73
5,10,40.94,33.46,44.09
6,12,43.31,36.22,46.46
7,14,45.67,38.98,48.82
8,16,48.82,42.52,51.97
9,18,51.97,46.06,55.12


## Notice how there isn't XS to XL! No worries. I cross checked with Mango's size guide and verified that 32.28 in Bust is XS and 40.94 is XL. 

In [44]:
df = df.iloc[1:]     
df_mango = df.iloc[0:5].reset_index(drop=True)  
df_mango

,Size,Bust,Waist,Hip
0,2,32.28,24.41,35.43
1,4,33.86,25.98,37.01
2,6,36.22,28.35,39.37
3,8,38.58,30.71,41.73
4,10,40.94,33.46,44.09


## Adding brand Mango in the column. 

In [87]:
df_mango['Brand'] = 'Mango'
df_mango

,Size,Bust,Waist,Hip,Brand,General Size
0,2,32.28,24.41,35.43,Mango,XS
1,4,33.86,25.98,37.01,Mango,S
2,6,36.22,28.35,39.37,Mango,M
3,8,38.58,30.71,41.73,Mango,L
4,10,40.94,33.46,44.09,Mango,XL


## Add the XS to XL. ( I cross checked with Mango's other size guide websites. We didn't scrape those websites because it's much harder. 

In [46]:
import numpy as np

n = len(df_mango)

df_mango['General Size'] = np.tile(['XS', 'S', 'M', 'L', 'XL'], n // 5 + 1)[:n]

df_mango['General Size'] = pd.Categorical(
    df_mango['General Size'],
    categories=['XS', 'S', 'M', 'L', 'XL'],
    ordered=True
)
df_mango

,Size,Bust,Waist,Hip,Brand,General Size
0,2,32.28,24.41,35.43,Mango,XS
1,4,33.86,25.98,37.01,Mango,S
2,6,36.22,28.35,39.37,Mango,M
3,8,38.58,30.71,41.73,Mango,L
4,10,40.94,33.46,44.09,Mango,XL


## Let's scrape Nasty Gal 

In [88]:
await page.goto("https://www.nastygal.com/pages/informational/size-guide")

<Response url='https://www.nastygal.com/pages/informational/size-guide' request=<Request url='https://www.nastygal.com/pages/informational/size-guide' method='GET'>>

In [89]:
import io
import pandas as pd

html = await page.content()
tables = pd.read_html(io.StringIO(html))
df = tables[0]
df

,Size (US/Canada),Bust (inches),Waist (inches),Hips (inches)
0,0/2,31-33,24-26,34-36
1,4/6,33-35,26-28,36-38
2,8/10,35-37,28-30,38-40
3,12/14,37-39,30-32,40-42
4,16/18,39-41,32-34,42-44


## Separating the multiple values in the cell. 

In [90]:
rows = []
for idx, row in df.iterrows():
    bust_parts = row['Bust (inches)'].split('-')
    waist_parts = row['Waist (inches)'].split('-')
    hips_parts = row['Hips (inches)'].split('-')
    
    row1 = [row['Size (US/Canada)'].split('/')[0] if '/' in str(row['Size (US/Canada)']) else row['Size (US/Canada)'],
            bust_parts[0], waist_parts[0], hips_parts[0]]
    
    row2 = [row['Size (US/Canada)'].split('/')[1] if '/' in str(row['Size (US/Canada)']) else row['Size (US/Canada)'],
            bust_parts[1], waist_parts[1], hips_parts[1]]
    
    rows.append(row1)
    rows.append(row2)

df_nastygal = pd.DataFrame(rows, columns=df.columns)
df_nastygal

,Size (US/Canada),Bust (inches),Waist (inches),Hips (inches)
0,0,31,24,34
1,2,33,26,36
2,4,33,26,36
3,6,35,28,38
4,8,35,28,38
5,10,37,30,40
6,12,37,30,40
7,14,39,32,42
8,16,39,32,42
9,18,41,34,44


## Dropping unnecessary columns. And renaming columns for future joining. 

df_nastygal= df_nastygal.rename(columns={
    'General Size': 'US Clothing Size',
    'Bust (inches)': 'Bust',
    'Waist (inches)': 'Waist',
    'Hips (inches)' : 'Hip'
})

df_nastygal = df_nastygal.drop(['Size (US/Canada)'], axis=1)

df_nastygal

## Notice that there isn't XS to XL again! I cross checked with another size guide within Nasty Gal's website. 

In [92]:
import numpy as np

n = len(df_nastygal)

sizes = ['XS', 'S', 'M', 'L', 'XL']
df_nastygal['General Size'] = np.repeat(sizes, 2) 

df_nastygal['General Size'] = np.tile(df_nastygal['General Size'], n // len(np.repeat(sizes, 2)) + 1)[:n]

df_nastygal['General Size'] = pd.Categorical(
    df_nastygal['General Size'],
    categories=sizes,
    ordered=True
)

df_nastygal['Brand']= 'Nasty Gal'
df_nastygal

,Bust,Waist,Hip,General Size,Brand
0,31,24,34,XS,Nasty Gal
1,33,26,36,XS,Nasty Gal
2,33,26,36,S,Nasty Gal
3,35,28,38,S,Nasty Gal
4,35,28,38,M,Nasty Gal
5,37,30,40,M,Nasty Gal
6,37,30,40,L,Nasty Gal
7,39,32,42,L,Nasty Gal
8,39,32,42,XL,Nasty Gal
9,41,34,44,XL,Nasty Gal


In [58]:
cols_to_convert = ['Bust', 'Waist', 'Hip',]

for col in cols_to_convert:
    df_nastygal[col] = pd.to_numeric(df_nastygal[col], errors='coerce')

## Let's scrape Fashion Nova

In [124]:
await page.goto("https://www.fashionnova.com/pages/size-guide")

<Response url='https://www.fashionnova.com/pages/size-guide' request=<Request url='https://www.fashionnova.com/pages/size-guide' method='GET'>>

In [125]:
import io
import pandas as pd

html = await page.content()
tables = pd.read_html(io.StringIO(html))
df = tables[0]
df

,0,1,2,3,4
0,Size,Bust,Waist,Hips,US/CAN
1,XS,32-33,24-25,35-36,0/1
2,S,34-35,26-27,37-38,3/5
3,M,36-37,28-29,39-40,7/9
4,L,38.5-40,30.5-32,41.5-43,11/13
5,XL,41 1/2,33 1/2,44 1/2,15
6,1X,44-45.5,37-38.5,47-48.5,14/16
7,2X,47-49,40-42,50-52,18/20
8,3X,51-53,44-46,54-56,22/24


## making the first row a column name

In [126]:
df = df.rename(columns=df.iloc[0]).iloc[1:].reset_index(drop=True)
df

,Size,Bust,Waist,Hips,US/CAN
0,XS,32-33,24-25,35-36,0/1
1,S,34-35,26-27,37-38,3/5
2,M,36-37,28-29,39-40,7/9
3,L,38.5-40,30.5-32,41.5-43,11/13
4,XL,41 1/2,33 1/2,44 1/2,15
5,1X,44-45.5,37-38.5,47-48.5,14/16
6,2X,47-49,40-42,50-52,18/20
7,3X,51-53,44-46,54-56,22/24


## Drop the US/CAN.

In [127]:
df = df.drop(['US/CAN'], axis=1)
df

,Size,Bust,Waist,Hips
0,XS,32-33,24-25,35-36
1,S,34-35,26-27,37-38
2,M,36-37,28-29,39-40
3,L,38.5-40,30.5-32,41.5-43
4,XL,41 1/2,33 1/2,44 1/2
5,1X,44-45.5,37-38.5,47-48.5
6,2X,47-49,40-42,50-52
7,3X,51-53,44-46,54-56


## Take " - " into account 

In [128]:
rows = []

for _, row in df.iterrows():
    bust_parts = row['Bust'].split('-')
    waist_parts = row['Waist'].split('-')
    hips_parts = row['Hips'].split('-')
    
    row1 = [row['Size'], bust_parts[0], waist_parts[0], hips_parts[0]]
    row2 = [row['Size'], bust_parts[1] if len(bust_parts)>1 else bust_parts[0],
            waist_parts[1] if len(waist_parts)>1 else waist_parts[0],
            hips_parts[1] if len(hips_parts)>1 else hips_parts[0]]
    
    rows.append(row1)
    rows.append(row2)

df_fashionnova = pd.DataFrame(rows, columns=df.columns)
df_fashionnova

,Size,Bust,Waist,Hips
0,XS,32,24,35
1,XS,33,25,36
2,S,34,26,37
3,S,35,27,38
4,M,36,28,39
5,M,37,29,40
6,L,38.5,30.5,41.5
7,L,40,32,43
8,XL,41 1/2,33 1/2,44 1/2
9,XL,41 1/2,33 1/2,44 1/2


## Rename Hips to Hip and only look from XS to XL. 

In [129]:
df_fashionnova.rename(columns={"Hips": "Hip"}, inplace=True)
df_fashionnova.rename(columns={"Size": "General Size"}, inplace=True)
df_fashionnova = df_fashionnova.iloc[0:10].reset_index(drop=True)  
df_fashionnova['Brand']= 'Fashion Nova'
df_fashionnova

,General Size,Bust,Waist,Hip,Brand
0,XS,32,24,35,Fashion Nova
1,XS,33,25,36,Fashion Nova
2,S,34,26,37,Fashion Nova
3,S,35,27,38,Fashion Nova
4,M,36,28,39,Fashion Nova
5,M,37,29,40,Fashion Nova
6,L,38.5,30.5,41.5,Fashion Nova
7,L,40,32,43,Fashion Nova
8,XL,41 1/2,33 1/2,44 1/2,Fashion Nova
9,XL,41 1/2,33 1/2,44 1/2,Fashion Nova


## Remove the 1/2 and make it all integers 

In [132]:
import fractions
def mixed_to_float(x):
    try:
        if pd.isna(x):
            return None
        return float(sum(fractions.Fraction(part) for part in x.split()))
    except:
        return None

# Apply to columns
for col in ['Bust', 'Waist', 'Hip']:
    df_fashionnova[col] = df_fashionnova[col].astype(str).apply(mixed_to_float)
df_fashionnova

,General Size,Bust,Waist,Hip,Brand
0,XS,32.0,24.0,35.0,Fashion Nova
1,XS,33.0,25.0,36.0,Fashion Nova
2,S,34.0,26.0,37.0,Fashion Nova
3,S,35.0,27.0,38.0,Fashion Nova
4,M,36.0,28.0,39.0,Fashion Nova
5,M,37.0,29.0,40.0,Fashion Nova
6,L,38.5,30.5,41.5,Fashion Nova
7,L,40.0,32.0,43.0,Fashion Nova
8,XL,41.5,33.5,44.5,Fashion Nova
9,XL,41.5,33.5,44.5,Fashion Nova


## Awesome. All of our data frames have the same column names. Now, it's time to make it one data frame. 

In [133]:
import pandas as pd

dfs = [df_fashionnova, df_nastygal, df_mango, df_amazon, df_target, df_garage]

columns_order = ['General Size', 'Bust', 'Waist', 'Hip', 'Brand']

dfs_reordered = [df[columns_order] for df in dfs]

df_all_brands = pd.concat(dfs_reordered, ignore_index=True)

df_all_brands

,General Size,Bust,Waist,Hip,Brand
0,XS,32.0,24.0,35.0,Fashion Nova
1,XS,33.0,25.0,36.0,Fashion Nova
2,S,34.0,26.0,37.0,Fashion Nova
3,S,35.0,27.0,38.0,Fashion Nova
4,M,36.0,28.0,39.0,Fashion Nova
5,M,37.0,29.0,40.0,Fashion Nova
6,L,38.5,30.5,41.5,Fashion Nova
7,L,40.0,32.0,43.0,Fashion Nova
8,XL,41.5,33.5,44.5,Fashion Nova
9,XL,41.5,33.5,44.5,Fashion Nova


In [135]:
cols_to_convert = ['Bust', 'Waist', 'Hip',]

for col in cols_to_convert:
    df_all_brands[col] = pd.to_numeric(df_all_brands[col], errors='coerce')

In [ ]:
df_all_brands.to_csv('../data/additional-brands.csv', index=False)